# DeepCube Earth System Data Cube functionalities 
This notebook is a demonstration of some functionalities of the Earth System Data Cube technology  developed in the context of Horizon2020 project DeepCube and prepared for the final review meeting.

The tools are developed in [Julia](https://julialang.org/), a high-performance and versatile scientific programming language. This Jupyter notebook must hence be run with a Julia kernel, preferably with Julia 1.10.0.

In [ ]:
using Pkg; Pkg.activate(".")

When running the notebook for the first time, after activating the environment in the cell above, the cell below needs to be run to instantiate the aforementioned environment in order to install all required packages and their dependencies.

In [ ]:
using Pkg; Pkg.instantiate()

The packages can then be loaded.

In [ ]:
using YAXArrays, Zarr
using OnlineStats: Mean, value, fit!, nobs
using YAXArrays.Cubes: cubesize, formatbytes

## Access data cube

For this demo, we rely on [mesogeos](https://zenodo.org/records/7741518), a Mediterranean data cube for the modelling & analysis of wildfires developed in DeepCube. It is stored on the cloud and directly accessible as an s3 bucket.

In [ ]:
ds = open_dataset("https://my-uc3-bucket.s3.gra.io.cloud.ovh.net/mesogeos.zarr")

In [ ]:
# show variables names
 print(keys(ds.cubes))

The dataset contains 30 variables, which all have dimensions x (longitude) and y (latitude). Most of them also have a time dimension.

We select one variable, `burned_areas`, to get information on the size of the cube and its chunking, that is, the way it is stored in compressed files.

In [ ]:
# get dimensions
println("Data cube size: $(size(ds.burned_areas))")
# get chunks
println("Number of chunks: $(size(ds.burned_areas.chunks))")

The dataset is chunked in a way most adapted for spatial analyses: each chunk contains data from one variable over its full spatial extent and only one time step.If we want to analyses more than one time step at a time, we have to load into memory as many chunks as time steps we want to analyse simultaneously. Even if our analysis focuses on a small spatial area, we have to load the full spatial extent into memory.

Hence, for temporal analyses, it is more advisable to rechunk the dataset to chunks with a smaller spatial extent but with more time steps. 

## Rechunking

The rechunking requires to first download the full dataset. It can then be done with YAXArrays.jl in two lines of code as shown below.

In [ ]:
# # DO NOT RUN
# dssub = ds[[:burned_areas,:net_ecosystem_exchange,:leaf_area_index]]

# dschunked = setchunks(dssub,target_chunks)

# savedataset(dschunked,path = "/Net/Groups/BGI/work_3/scratch/fgans/DeepCube/UC3Cube_rechunked2.zarr", max_cache=1e9, backend = :zarr,overwrite = false)
# # max_cache determines the amount of memory to be used for rechunking, the larger this is, the faster the rechunking will go
# # backend can be either :zarr or :netcdf
# # setting overwrite=true will delete any existing dataset
# # setting append=true will append the newly chunked variables to an existing data cube


Once the dataset has been rechunked to serve our purpose, we can access it and perform our temporal analysis.

## Temporal analysis

Our demo aims to analyse to which extent the mean of a selection of variables are different inside and outside burnt areas. Therefore, we compute the difference between a variable mean over time and over a spatial moving window `(7, 7)` inside burnt areas and in their direct surroundings, that is, outside the burnt area but within the spatial window and over the duration of the fire event. Since most areas are never burnt, we do not need to compute the means at every location, but only where there was a fire. Hence, we first identify "blobs" of contiguous burnt areas in these spatial windows and process them one at a time. That is, we load in memory just the chunks intersecting the spatial window and the timesteps at which the area was burnt.

In [ ]:
# Open the time series cube
ds = open_dataset("/Net/Groups/BGI/work_3/scratch/fgans/DeepCube/UC3Cube_rechunked2.zarr");

We select six variables as potential predictors for our analysis of burnt areas: night and land surface temperature, elevation, forest and grass cover fractions and distance to nearest road.

The total size of our uncompressed subdataset amounts to almost 1TB.

In [ ]:
# Select variables of interest
burned_area = ds.burned_areas;
preds = ("lst_night", "lst_day","dem", "lc_forest", "lc_grassland", "roads_distance")
possible_predictors = map(i->ds[Symbol(i)],preds);

# Total uncompressed data size:
formatbytes(sum(cubesize,(burned_area,possible_predictors...)))

In the following code lines, we define the dimensions over which we are going to run our analysis, that is, a `(7, 7, 6026)` moving window centred on our output grid cell, for our target and predictor variables (unless the latter does not have a time dimension). The output dimensions are defined as the predictor variables. Because we used `MovingWindow` in the input dimensions, the output will have `"x"` and `"y"` dimensions as the input dataset.

In [ ]:
indims_burnedarea = InDims(MovingWindow("x",3,3), MovingWindow("y",3,3), "Time", window_oob_value = 0.f0)
indims_predictors = map(possible_predictors) do p
    td = ndims(p) == 3 ? ("Time",) : ()
    InDims(MovingWindow("x",3,3), MovingWindow("y",3,3), td..., window_oob_value = 0.f0)
end

outdims = OutDims(
    Dim{:Variable}(collect(preds)), 
    outtype = Float32, 
    backend=:zarr,
    path = "./output.zarr", 
    overwrite=true
)


We set up the job to be run on the cluster. The analysis is run on the MPI-BGC cluster but could be run on any cloud computing facility supporting Julia or docker containers.

Therefore, we load two new Julia packages: `SlurmManager` to interact with the Slurm cluster and `Distributed`, which supports distributed computing. For some reason, on our partition for big data, the workers need to be spawned one at a time.

In [ ]:
n_workers = 20
threads_per_worker = 16
# Get 20 workers with 16 cpus per worker
using ClusterManagers: SlurmManager
using Distributed
map(1:n_workers) do i
    Threads.@spawn begin
        addprocs(
            SlurmManager(1,fill(2.0,10)),
            partition="big",
            mem_per_cpu="16GB",
            time="00:30:00",
            cpus_per_task=threads_per_worker,
            exeflags=`--project=$(@__DIR__) -t 32 --heap-size-hint=8GB`
            )
    end
end

Once the workers are spawned, we load the required packages and functions on all of them. [windowfire.jl](windowfire.jl) is the script that contains the functions `fire_boundaries_window!` and `process_event`, which are at the core of our analysis.

In [ ]:
# Load code everywhere
@everywhere begin
    using YAXArrays, Zarr
    using OnlineStats: Mean, value, fit!, nobs
    include("windowfire.jl")
    Zarr.Blosc.set_num_threads(threads_per_worker)
end

The `fire_boundaries_window!` function is applied in parallel on the moving windows thanks to the `YAXArrays` function `mapCube`.

In [ ]:
mapCube(
    fire_boundaries_window!, 
    (burned_area, possible_predictors...);
    indims = (indims_burnedarea, indims_predictors...), 
    outdims = outdims,
    max_cache=2e9,
)

Once the above code has been executed, the workers can be closed and the results can be plotted.

In [ ]:
rmprocs(workers())

## Visualisation

The output data cube contains the differences between the averages inside and outside burnt areas in the direct surrounding (`(7, 7)` window) for the six possible predictor variables. It has no time dimension, only x and y.

We can create maps of the output after loading plotting packages.

In [ ]:
using CairoMakie, Makie, GeoMakie, PerceptualColourMaps
using YAXArrays, Zarr
using Statistics

if !isdir("./figs")
    mkdir("./figs")
end

# output of moving windo demo
ds = open_dataset("/Net/Groups/BGI/people/fgans/DeepCube/FinalESDCDEmo/output.zarr/")

preds = ("lst_night", "lst_day","dem", "lc_forest", "lc_grassland", "roads_distance")
possible_predictors = map(i->ds[Symbol(i)],preds);

# units of the predictors, used in the legend of the maps
units = ("Temperature [Kelvin]","Temperature [Kelvin]","Elevation [m]", "Forest cover [%]", "Grass cover [%]", "Distance to nearest road [km]")

function myfig!(fig, lon, lat, data)
    ax = GeoAxis(fig[1,1], limits=(extrema(lon), extrema(lat)))
    s = surface!(ax, lon, lat, data; 
        colorrange=(-maximum(abs.(q)), maximum(abs.(q))),
        colormap = cmap("CBD2"),#:bluesreds, 
        # shading=`NoShading`,
    )
    cl=lines!(ax, GeoMakie.coastlines(), color = :black, linewidth=0.85)
    # translate!(cl, 0, 0, 1000)
    Colorbar(fig[1,2], s, label = units[i])
    Label(fig[1, 1:end, Top()], "Difference in average $(preds[i]) inside and outside burned areas", fontsize=18, padding=(0, 6, 8, 0))

    return(fig)
end


In [ ]:
i=1
    fig = Figure(;size=(1200,600));
    c = possible_predictors[i]
    lon = lookup(c, :x)
    lat = lookup(c, :y)
    data = c.data[:,:];

    @show q = Statistics.quantile(filter(i->!ismissing(i) && !isnan(i), data),(0.1,0.5,0.9))

    fig = myfig!(fig, lon, lat, data)
    # save("./figs/$(preds[i]).png", fig)


In [ ]:

    # subset cube 
    sc = c[x = 19 .. 26, y = 36 .. 42]
    slon = lookup(sc, :x)
    slat = lookup(sc, :y)
    sdata = sc.data[:,:];
    fig = Figure(;size=(500,500));
    fig = myfig!(fig, slon, slat, sdata)
    # save("./figs/$(preds[i])_zoom.png", fig)
